In [ ]:
from analysis.analysis import *
import matplotlib.pyplot as plt
import os

root = "diff_geoms"
gid = 6
default_LC_gid =10

Read the raw sim data for a grid ID

In [ ]:
data_root = f"../_server/sim_data/{root}"

grid_df = read_fulltake(
    os.path.join(data_root, f"fulltake_gid{gid}.parquet"), scale_position=True
)

print("# of runs:", grid_df.rid.unique().size)
cell_config = yaml.safe_load(
    open(os.path.join(f"../configs/IM/grid_id{gid}/cell.yaml"), "rb")
)
simbox_config = yaml.safe_load(
    open(os.path.join(f"../configs/IM/grid_id{gid}/simbox.yaml"), "rb")
)

---

In [ ]:
simbox_config["substrate"]

In [ ]:
from substrate.substrates import Substrate

chi = Substrate(200, 50, 0.2).mixed_two_state_sub(simbox_config["substrate"]["basin_dims"], simbox_config["substrate"]["bridge_dim"])

In [ ]:
xy = grid_df[["x", "y"]].values

fig, ax = plt.subplots(1, 2, figsize=(6, 3))
ax[0].imshow(chi, cmap="Greys_r", alpha=0.5, extent=[0, 50, 0, 50], origin="lower")
ax[0].set_xlim((25-12, 25+12))
ax[0].set_ylim((25-12, 25+12))
ax[1].hist(xy[:, 0], bins=50, density=True)
ax[1].set_xlabel(r"$x$ [$\mu$m]")
fig.tight_layout()
plt.show()

In [ ]:
np.where(xy[:, 0] <= (150 - simbox_config["substrate"]["bridge_dim"][0]/2))[0].size / xy[:, 0].size

In [ ]:
np.where(xy[:, 0] >= (150 + simbox_config["substrate"]["bridge_dim"][0]/2))[0].size / xy[:, 0].size

In [ ]:
0.4754939236111111 + 0.3425390625